In [1]:
# import dependencies & check tf version
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.3.0


In [2]:
# load mnist dataset & normalize data 
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
# add a channels dimension behind ((60000, 28, 28) -> (60000, 28, 28, 1))
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [4]:
# prepare dataset (batch + shuffle*)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
# build ML model using Keras Model subclassing API
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [6]:
# define objective function (aka. loss / cost / error function) & optimizer for training
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
# select metrics to measure Loss & Accuracy (accumulate over epochs *can be reset every epoch)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
# def function to train model
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [9]:
# def function to test / val model
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
# train & validate model
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.13470807671546936, Accuracy: 95.91000366210938, Test Loss: 0.06623464822769165, Test Accuracy: 97.7199935913086
Epoch 2, Loss: 0.043361034244298935, Accuracy: 98.67500305175781, Test Loss: 0.05451316386461258, Test Accuracy: 98.1500015258789
Epoch 3, Loss: 0.0239400714635849, Accuracy: 99.20500183105469, Test Loss: 0.0552385151386261, Test Accuracy: 98.48999786376953
Epoch 4, Loss: 0.014314922504127026, Accuracy: 99.53666687011719, Test Loss: 0.0518961176276207, Test Accuracy: 98.36000061035156
Epoch 5, Loss: 0.009225042536854744, Accuracy: 99.66666412353516, Test Loss: 0.06733538210391998, Test Accuracy: 98.30999755859375
